In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [246]:
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from tensorflow.keras.models import Sequential

In [253]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [94]:
data = pd.read_csv('../raw_data/emotion_data.csv')

In [95]:
data.head()

,Emotion,Text
0,sad,Layin n bed with a headache ughhhh...waitin o...
1,sad,Funeral ceremony...gloomy friday...
2,joy,wants to hang out with friends SOON!
3,worry,Re-pinging @ghostridah14: why didn't you go to...
4,sad,"I should be sleep, but im not! thinking about ..."


In [97]:
data['Emotion'].value_counts()

anger      15872
joy        14168
worry      11786
neutral    10212
sad         9233
Name: Emotion, dtype: int64

In [102]:
data.shape

(61271, 2)

In [272]:
X = data[['Text']]
y = data[['Emotion']]

In [273]:
from sklearn.model_selection import train_test_split

In [274]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.utils import simple_preprocess

In [275]:
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [276]:
def clean_data(dataframe):
    """
    clean and preprocess data
    """
    
    data = dataframe.copy()

    # Lowercase text
    data['clean_text'] = data['Text'].apply(
        lambda x: x.lower()
        )

    # Strip whitespace
    data['clean_text'] = data['clean_text'].apply(
        lambda x: x.strip()
        )

    # Remove numbers
    data['clean_text'] = data['clean_text'].apply(
        lambda x: ''.join(let for let in x if not let.isdigit())
        )

    # Remove punctuation
    data['clean_text'] = data['clean_text'].apply(
        lambda x: ''.join(let for let in x if not let in string.punctuation)
        )

    # Tokenization with nltk
    data['clean_text'] = data['clean_text'].apply(
        lambda x: word_tokenize(x)
    )
    
    # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     data['clean_text'] = data['clean_text'].apply(
#         lambda x: [word for word in x if word not in stop_words]
#         )

    # Lemmatizing with nltk
    lemmatizer = WordNetLemmatizer()
    data['clean_text'] = data['clean_text'].apply(
        lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x)
        )
    
    # Tokenizing text
    data['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in data['clean_text']] 

    # Return data
    return data

In [277]:
emotions_df = clean_data(data)

In [313]:
# Train Test Split Function
def split_train_test(df, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(df[['tokenized_text']], 
                                                        df[['Emotion']], 
                                                        test_size=test_size, 
                                                        random_state=15)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_train_test(emotions_df, test_size=0.3)

In [279]:
X_train.head()

,tokenized_text
32200,"[yeah, it, lot, better, than, mine]"
1030,"[thomasfiss, think, you, should, come, tomarro..."
56436,"[ive, been, up, since, hate, wen, cant, sleep,..."
29605,"[sure, now, you, are, talking]"
28747,"[sometimes, ive, thought, ll, never, forget, w..."


Training data stats

In [280]:
all_training_words = [word for tokens in X_train["tokenized_text"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in X_train["tokenized_text"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

531063 words total, with a vocabulary size of 39182
Max sentence length is 201


Testing data stats

In [281]:
all_test_words = [word for tokens in X_test["tokenized_text"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in X_test["tokenized_text"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

226316 words total, with a vocabulary size of 22205
Max sentence length is 152


In [282]:
from gensim import models

In [314]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(X_train["tokenized_text"].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train["tokenized_text"].tolist())
train_word_index = tokenizer.word_index
MAX_SEQUENCE_LENGTH = 201
print("Found %s unique tokens." % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)

Found 39182 unique tokens.


In [285]:
EMBEDDING_DIM = 300

In [286]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
 train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(39183, 300)


In [287]:
test_sequences = tokenizer.texts_to_sequences(X_test["tokenized_text"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [270]:
np.array(test_sequences).shape

/Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(18382,)

In [315]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [322]:
label_names = ['anger', 'joy', 'worry', 'neutral', 'sad']
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 201)]        0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 201, 300)     11754900    input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 200, 200)     120200      embedding_12[0][0]               
__________________________________________________________________________________________________
conv1d_28 (Conv1D)              (None, 199, 200)     180200      embedding_12[0][0]               
_______________________________________________________________________________________

In [289]:
# Prepare mapping for the sentiment
sentiment_coding = {'anger': 0 , 'joy': 4, 'worry': 2, 'sad': 1 , 'neutral': 3}

# apply mapping
y_train_coded = y_train['Emotion'].map(sentiment_coding)

# Transform the numbers to categories
y_train_cat = to_categorical(y_train_coded)

# Prepare y_test
y_test_coded= y_test['Emotion'].map(sentiment_coding)
y_test_coded.value_counts()
y_test_cat = to_categorical(y_test_coded)

In [290]:
y_train_cat

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [318]:
num_epochs = 3
batch_size = 34
X_train_model = train_cnn_data
y_tr = y_train_cat

In [319]:
len(X_train)

42889

In [357]:
X_train_model.shape

(42889, 201)

In [320]:
y_tr

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [323]:
history = model.fit(X_train_model, y_tr,
                 validation_split=0.3,
                 batch_size=batch_size,
                 epochs=num_epochs,
                 verbose=1
                )

Epoch 1/3
883/883 [==============================] - 575s 651ms/step - loss: 1.3228 - accuracy: 0.4402 - val_loss: 1.1803 - val_accuracy: 0.5248
Epoch 2/3
883/883 [==============================] - 429s 486ms/step - loss: 1.1290 - accuracy: 0.5442 - val_loss: 1.0963 - val_accuracy: 0.5589
Epoch 3/3
883/883 [==============================] - 421s 477ms/step - loss: 1.0086 - accuracy: 0.5961 - val_loss: 1.0759 - val_accuracy: 0.5702


In [326]:
# Train Test Split Function
def split_train_test(df, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(df[['tokenized_text']], 
                                                        df[['Emotion']], 
                                                        test_size=test_size, 
                                                        random_state=15)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_train_test(emotions_df, test_size=0.3)

In [335]:
X_train

,tokenized_text
32200,"[yeah, it, lot, better, than, mine]"
1030,"[thomasfiss, think, you, should, come, tomarro..."
56436,"[ive, been, up, since, hate, wen, cant, sleep,..."
29605,"[sure, now, you, are, talking]"
28747,"[sometimes, ive, thought, ll, never, forget, w..."
...,...
49015,"[when, passed, the, ba, exam, with, st, class,..."
2693,"[labelladiva, are, you, blogging, yet, miss, y..."
8076,"[wish, wa, going, to, be, with, my, st, girl, ..."
52981,"[laccboner, man, can, cut, my, hair, now, mean..."


In [343]:
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convert_sentences(X):
    return [sentence.split(' ') for sentence in X]

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

def embedding_pipeline(word2vec, X):
    # Step #2: List of words to list of vectors
#     X = convert_sentences(X)
    # Step #3: Sentences to list of words
    X = embedding(word2vec, X)
    # Step #4: Pad the inputs
    X = pad_sequences(X, dtype='float32', padding='post', value= -1000)
    
    return X

In [339]:
from tensorflow.keras.preprocessing.text import Tokenizer

texts = emotions_df.clean_text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequence_train = tokenizer.texts_to_sequences(X_train['tokenized_text'])
sequence_test = tokenizer.texts_to_sequences(X_test['tokenized_text'])

index_of_words = tokenizer.word_index

# vacab size is number of unique words + reserved 0 index for padding
vocab_size = len(index_of_words) + 1

In [340]:
len(sequence_train)

42889

In [338]:
sequence_train

[[146, 7, 147, 165, 200, 235],
 [3663,
  79,
  5,
  140,
  111,
  16760,
  13,
  46,
  46,
  46,
  222,
  16761,
  3,
  502,
  16762,
  111],
 [153,
  87,
  43,
  367,
  206,
  2893,
  84,
  199,
  4458,
  15838,
  1032,
  6,
  4131,
  33,
  2,
  238],
 [144, 50, 5, 31, 396],
 [545,
  153,
  170,
  374,
  178,
  618,
  44,
  390,
  7,
  35,
  111,
  81,
  6,
  106,
  442,
  110,
  110,
  2078,
  110,
  5055,
  6,
  3371],
 [75, 135, 13, 3219],
 [8712, 45095, 20, 5740, 378, 45096, 15, 1094, 11, 2307],
 [23,
  13,
  1029,
  18,
  15686,
  20,
  1270,
  49,
  9,
  8,
  62,
  121,
  13,
  967,
  10,
  59,
  236,
  6,
  334,
  59,
  204,
  54,
  15,
  13,
  46,
  333,
  18,
  104,
  101],
 [48, 1889],
 [1201, 137, 20, 6552, 50, 18, 3561, 6, 49753],
 [23, 13, 10, 172, 1967, 1486, 6, 34, 801, 191, 297, 356, 10, 2, 660, 1331],
 [60, 25, 19, 2079, 3099],
 [13085, 645, 101, 32111, 7741, 14, 641, 989, 163, 942, 377, 71, 11, 2, 1747],
 [14, 538, 323, 10, 41, 943, 202],
 [7, 2, 172, 120, 12, 97, 18

In [344]:
# Step #1: Train a word2vec - with X_train token so it takes into account the words and not the letters 
word2vec = Word2Vec(sentences=X_train['tokenized_text'], size=60, min_count=10, window=10)

X_train_pad = embedding_pipeline(word2vec, X_train['tokenized_text'])

X_test_pad = embedding_pipeline(word2vec, X_test['tokenized_text'])

X_train_pad.shape

(42889, 184, 60)

In [345]:
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

In [ ]:
X_test_pad.shape

In [346]:
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

Number of labels in train set {'anger': 11112, 'joy': 9890, 'neutral': 7145, 'sad': 6450, 'worry': 8292}


In [356]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dense , Flatten   

embed_num_dims = 201
max_seq_len = 50
# vocab_size

# # Embedding layer before the actaul BLSTM 
# embed_layer = Embedding(vocab_size,
#                          embed_num_dims,
#                          input_length = max_seq_len,
#                          trainable=False)

# def cnn_model():
#     model = Sequential()
#     model.add(embed_layer)
#     model.add(Conv1D(filters = 3,input_shape=(215,60),kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D())
#     model.add(Flatten())
    
#     model.add(Dense(5, activation='softmax'))
    
#     model.compile(loss = 'categorical_crossentropy', 
#                   optimizer = 'adam', 
#                   metrics = ['accuracy'])
#     return model

# model = cnn_model()    
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 201)]        0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 201, 300)     11754900    input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_47 (Conv1D)              (None, 200, 200)     120200      embedding_17[0][0]               
__________________________________________________________________________________________________
conv1d_48 (Conv1D)              (None, 199, 200)     180200      embedding_17[0][0]               
______________________________________________________________________________________

In [349]:
print(X_train_pad[6].shape)
y_train_cat

(184, 60)


array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [351]:
from tensorflow.keras.callbacks import EarlyStopping
batch_size = 32
epochs = 30

es = EarlyStopping(patience=15, restore_best_weights=True)

history = model.fit(X_train_pad, y_train_cat,
                 validation_split=0.3,
                 batch_size=batch_size,
                 epochs=epochs,
                 callbacks=[es],
                 verbose=1
                )

Epoch 1/30


ValueError: in user code:

    /Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/lukemoberly/.pyenv/versions/3.7.7/envs/lewagon_project/lib/python3

In [358]:
X_train_pad

array([[[-3.61902453e-02,  7.54565522e-02,  8.32705915e-01, ...,
         -1.30162358e-01, -6.44590557e-02, -2.38692731e-01],
        [-6.36480469e-03,  3.80894154e-01, -1.76049367e-01, ...,
         -2.84291029e-01, -2.06274882e-01, -5.18095851e-01],
        [-2.47367963e-01, -1.76975667e-01,  5.22972941e-01, ...,
         -3.68743896e-01,  3.54429394e-01,  9.73016083e-01],
        ...,
        [-1.00000000e+03, -1.00000000e+03, -1.00000000e+03, ...,
         -1.00000000e+03, -1.00000000e+03, -1.00000000e+03],
        [-1.00000000e+03, -1.00000000e+03, -1.00000000e+03, ...,
         -1.00000000e+03, -1.00000000e+03, -1.00000000e+03],
        [-1.00000000e+03, -1.00000000e+03, -1.00000000e+03, ...,
         -1.00000000e+03, -1.00000000e+03, -1.00000000e+03]],

       [[-3.67954485e-02, -8.62792209e-02,  1.93148162e-02, ...,
         -1.72100328e-02, -4.60774079e-03,  4.10171710e-02],
        [-2.83943474e-01,  3.68072242e-01,  1.36217833e+00, ...,
         -2.52310578e-02, -4.28110272e